# Notebook de modelisation

In [ ]:
# installation des dependances avec uv
# !uv pip install scikit-learn numpy hdbscan

## Imports des libs

In [ ]:
import pandas as pd
import numpy as np
from sklearn import cluster, metrics, mixture
from sklearn.datasets import make_blobs
import time
import hdbscan  # Pour HDBSCAN
from joblib import Parallel, delayed

## Fonction d'evaluation d'un modèle de clustering

In [ ]:
def evaluate_algorithm(name, algorithm, data):
    start_time = time.time()

    # Fit et prédire les labels
    labels = algorithm.fit_predict(data)
    elapsed_time = time.time() - start_time

    # Calcul des scores
    silhouette_score = metrics.silhouette_score(data, labels) if len(set(labels)) > 1 else None
    davies_bouldin_score = metrics.davies_bouldin_score(data, labels) if len(set(labels)) > 1 else None
    calinski_harabasz_score = metrics.calinski_harabasz_score(data, labels) if len(set(labels)) > 1 else None

    return {
        "Algorithm": name,
        "Time (s)": elapsed_time,
        "Silhouette Score": silhouette_score,
        "Davies-Bouldin Score": davies_bouldin_score,
        "Calinski-Harabasz Score": calinski_harabasz_score
    }

# Benchmark des models

Les modèles de clustering dispo avec [sklearn](https://scikit-learn.org/stable/modules/clustering.html) et son tableau comparatif

## Liste des algorithmes à comparer

In [ ]:

n_cluster_value = 3
min_sample_value = 1000

model_list = {
    'kmeans': cluster.KMeans(n_clusters=n_cluster_value),
    'minibatchkmeans': cluster.MiniBatchKMeans(n_clusters=n_cluster_value),
    'hdbscan': hdbscan.HDBSCAN(min_cluster_size=min_sample_value),
    'gmm': mixture.GaussianMixture(n_components=n_cluster_value),
    'birch': cluster.Birch(n_clusters=n_cluster_value),
    'agglomerative': cluster.AgglomerativeClustering(n_clusters=n_cluster_value),
    'dbscan': cluster.DBSCAN(eps=0.5, min_samples=min_sample_value),
    'optics': cluster.OPTICS(min_samples=min_sample_value),
    'meanshift': cluster.MeanShift(),
}

## Benchmark RFM

In [ ]:
# import des données
rfm = pd.read_excel('RFM.xlsx')
rfm = rfm.drop(['customer_unique_id', 'Segment'], axis=1).dropna()
rfm.head()

In [ ]:
# On prend 10% des données pour gagner du temps d'execution
rfm_sample = rfm.sample(frac=0.1, random_state=42)
print(rfm_sample.shape)

### Entrainement des modèle (Utilisation du multithreading)

In [ ]:
results = []

# Exécution en parallèle avec joblib
results = Parallel(n_jobs=-1)(delayed(evaluate_algorithm)(name, algorithm, rfm_sample)for name, algorithm in model_list.items())

# Création d'un DataFrame pour afficher les résultats
results_df = pd.DataFrame(results)

In [ ]:
# Silhouette Score : Plus il est élevé, mieux c'est
# Davies-Bouldin Score : Plus il est faible, mieux c'est 
# Calinski-Harabasz Score : Plus il est élevé, mieux c'est

results_df.sort_values(by=['Silhouette Score', 'Davies-Bouldin Score', 'Calinski-Harabasz Score'], ascending=[False, True, False])

#### Find the best number of cluster (Plot metrics per number of cluster)

In [ ]:
import matplotlib.pyplot as plt
from sklearn import cluster
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import numpy as np

def plot_metrics_for_each_model(data, max_k=10):
    metrics = {
        'kmeans': {'inertia': [], 'silhouette': [], 'davies_bouldin': [], 'calinski_harabasz': []},
        'birch': {'inertia': [], 'silhouette': [], 'davies_bouldin': [], 'calinski_harabasz': []},
        'agglomerative': {'silhouette': [], 'davies_bouldin': [], 'calinski_harabasz': []},
    }

    k_values = range(2, max_k + 1)

    for k in k_values:
        # KMeans
        kmeans = cluster.KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(data)
        metrics['kmeans']['inertia'].append(kmeans.inertia_)
        if len(set(labels)) > 1:
            metrics['kmeans']['silhouette'].append(silhouette_score(data, labels))
            metrics['kmeans']['davies_bouldin'].append(davies_bouldin_score(data, labels))
            metrics['kmeans']['calinski_harabasz'].append(calinski_harabasz_score(data, labels))

        # Birch
        birch = cluster.Birch(n_clusters=k)
        labels = birch.fit_predict(data)
        if len(set(labels)) > 1:
            metrics['birch']['inertia'].append(birch.subcluster_centers_.shape[0])  # Approximation of inertia
            metrics['birch']['silhouette'].append(silhouette_score(data, labels))
            metrics['birch']['davies_bouldin'].append(davies_bouldin_score(data, labels))
            metrics['birch']['calinski_harabasz'].append(calinski_harabasz_score(data, labels))

        # Agglomerative Clustering
        agg = cluster.AgglomerativeClustering(n_clusters=k)
        labels = agg.fit_predict(data)
        if len(set(labels)) > 1:
            metrics['agglomerative']['silhouette'].append(silhouette_score(data, labels))
            metrics['agglomerative']['davies_bouldin'].append(davies_bouldin_score(data, labels))
            metrics['agglomerative']['calinski_harabasz'].append(calinski_harabasz_score(data, labels))

    # Tracé des métriques pour chaque modèle
    models = ['kmeans', 'birch', 'agglomerative']
    for model in models:
        plt.figure(figsize=(15, 10))

        # Elbow Method
        plt.subplot(2, 2, 1)
        if 'inertia' in metrics[model] and len(metrics[model]['inertia']) > 0:
            plt.plot(k_values, metrics[model]['inertia'], 'bo-', markersize=4)
            plt.xlabel('Number of clusters, k')
            plt.ylabel('Inertia')
            plt.title(f'{model.capitalize()} - Elbow Method')
            plt.grid(True)
        else:
            plt.title(f'{model.capitalize()} - Elbow Method (No Data)')

        # Silhouette Score
        plt.subplot(2, 2, 2)
        if len(metrics[model]['silhouette']) > 0:
            plt.plot(k_values, metrics[model]['silhouette'], 'go-', markersize=4)
            plt.xlabel('Number of clusters, k')
            plt.ylabel('Silhouette Score')
            plt.title(f'{model.capitalize()} - Silhouette Score')
            plt.grid(True)
        else:
            plt.title(f'{model.capitalize()} - Silhouette Score (No Data)')

        # Davies-Bouldin Score
        plt.subplot(2, 2, 3)
        if len(metrics[model]['davies_bouldin']) > 0:
            plt.plot(k_values, metrics[model]['davies_bouldin'], 'ro-', markersize=4)
            plt.xlabel('Number of clusters, k')
            plt.ylabel('Davies-Bouldin Score')
            plt.title(f'{model.capitalize()} - Davies-Bouldin Score')
            plt.grid(True)
        else:
            plt.title(f'{model.capitalize()} - Davies-Bouldin Score (No Data)')

        # Calinski-Harabasz Score
        plt.subplot(2, 2, 4)
        if len(metrics[model]['calinski_harabasz']) > 0:
            plt.plot(k_values, metrics[model]['calinski_harabasz'], 'mo-', markersize=4)
            plt.xlabel('Number of clusters, k')
            plt.ylabel('Calinski-Harabasz Score')
            plt.title(f'{model.capitalize()} - Calinski-Harabasz Score')
            plt.grid(True)
        else:
            plt.title(f'{model.capitalize()} - Calinski-Harabasz Score (No Data)')

        plt.tight_layout()
        plt.suptitle(f'{model.capitalize()} Clustering Metrics', fontsize=16)
        plt.show()

# Exemple d'utilisation avec des données fictives
# data = ... (votre jeu de données)
plot_metrics_for_each_model(rfm_sample)


Correspondance avec les sagments defini dans l'EDA
segment_order = ['Champions', 'Loyaux', 'Loyalistes potentiels', 'À réactiver', 'À risque', 'Perdus']

3 Correspond au 3 persona les plus presents et 6 correspond a tous les personas !

![{A84F2560-6E5B-4FE6-8987-0BEFCC5E62EC}.png](attachment:{A84F2560-6E5B-4FE6-8987-0BEFCC5E62EC}.png)

#### Visualisation des clusters

##### Vue 2D

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Supposons que 'rfm_sample' est votre rfm_sampleFrame RFM
# rfm_sample = pd.rfm_sampleFrame(...)

# Appliquer KMeans
kmeans = KMeans(n_clusters=3, random_state=42)
rfm_sample['Cluster'] = kmeans.fit_predict(rfm_sample[['R', 'F', 'M']])

# Analyser les centres des clusters
centers = pd.DataFrame(kmeans.cluster_centers_, columns=['R', 'F', 'M'])
# print("Centres des clusters :")
# print(centers)

# Calculer les statistiques descriptives pour chaque cluster
cluster_stats = rfm_sample.groupby('Cluster')[['R', 'F', 'M']].agg(['mean', 'median', 'std'])
# print("Statistiques descriptives des clusters :")
# print(cluster_stats)

# Visualiser les clusters
plt.figure(figsize=(15, 5))

# Graphique Récence vs Fréquence
plt.subplot(1, 3, 1)
for cluster in range(3):
    plt.scatter(rfm_sample[rfm_sample['Cluster'] == cluster]['R'], rfm_sample[rfm_sample['Cluster'] == cluster]['F'], label=f'Cluster {cluster}')
plt.xlabel('Récence')
plt.ylabel('Fréquence')
plt.title('Récence vs Fréquence')
plt.legend()

# Graphique Fréquence vs Monetary
plt.subplot(1, 3, 2)
for cluster in range(3):
    plt.scatter(rfm_sample[rfm_sample['Cluster'] == cluster]['F'], rfm_sample[rfm_sample['Cluster'] == cluster]['M'], label=f'Cluster {cluster}')
plt.xlabel('Fréquence')
plt.ylabel('Monetary')
plt.title('Fréquence vs Monetary')
plt.legend()

# Graphique Récence vs Monetary
plt.subplot(1, 3, 3)
for cluster in range(3):
    plt.scatter(rfm_sample[rfm_sample['Cluster'] == cluster]['R'], rfm_sample[rfm_sample['Cluster'] == cluster]['M'], label=f'Cluster {cluster}')
plt.xlabel('Récence')
plt.ylabel('Monetary')
plt.title('Récence vs Monetary')
plt.legend()

plt.tight_layout()
plt.show()


##### Vue 3D

In [ ]:
import plotly.express as px
import pandas as pd

# Supposons que 'data' est votre DataFrame RFM avec une colonne 'Cluster'
# data = pd.DataFrame(...)

# Créer un graphique 3D avec Plotly
fig = px.scatter_3d(rfm_sample, x='R', y='F', z='M', color='Cluster',
                    labels={'R': 'Récence', 'F': 'Fréquence', 'M': 'Monetary', 'Cluster': 'Cluster'},
                    title='Clustering RFM en 3D')

# Afficher le graphique
fig.show()


## Benchmark RFMLS

In [ ]:
# import des données
rfmls = pd.read_excel('RFMLS.xlsx')
rfmls = rfmls.drop(['customer_unique_id', 'Unnamed: 0'], axis=1).dropna()
rfmls.head()

In [ ]:
# On prend 10% des données pour gagner du temps d'execution
rfmls_sample = rfmls.sample(frac=0.1, random_state=42)
print(rfmls_sample.shape)

In [ ]:
import pandas as pd
from sklearn.feature_extraction import FeatureHasher
from sklearn.decomposition import PCA

# Initialisation du FeatureHasher
hasher = FeatureHasher(n_features=50, input_type='string')

# Transformation de la colonne 'L' en une liste de listes
hashed_features = hasher.transform([[city] for city in rfmls_sample['L']])

# Initialisation du PCA
pca = PCA(n_components=1)  # Réduction à une seule dimension

# Application du PCA sur les caractéristiques hachées
hashed_features_pca = pca.fit_transform(hashed_features.toarray())

# Ajout de la composante principale au DataFrame original
rfmls_sample['L_pca'] = hashed_features_pca

print(rfmls_sample)


### Entrainement des modèle (Utilisation du multithreading)

In [ ]:
rfmls_sample_train = rfmls_sample.drop('L', axis=1)

In [ ]:
results = []

# Exécution en parallèle avec joblib
results = Parallel(n_jobs=-1)(delayed(evaluate_algorithm)(name, algorithm, rfmls_sample_train)for name, algorithm in model_list.items())

# Création d'un DataFrame pour afficher les résultats
results_df = pd.DataFrame(results)

In [ ]:
# Silhouette Score : Plus il est élevé, mieux c'est
# Davies-Bouldin Score : Plus il est faible, mieux c'est 
# Calinski-Harabasz Score : Plus il est élevé, mieux c'est

results_df.sort_values(by=['Silhouette Score', 'Davies-Bouldin Score', 'Calinski-Harabasz Score'], ascending=[False, True, False])

#### Find the best number of cluster (Plot metrics per number of cluster)

In [ ]:
import matplotlib.pyplot as plt
from sklearn import cluster
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import numpy as np

def plot_metrics_for_each_model(data, max_k=10):
    metrics = {
        'kmeans': {'inertia': [], 'silhouette': [], 'davies_bouldin': [], 'calinski_harabasz': []},
        'birch': {'inertia': [], 'silhouette': [], 'davies_bouldin': [], 'calinski_harabasz': []},
        'agglomerative': {'silhouette': [], 'davies_bouldin': [], 'calinski_harabasz': []},
    }

    k_values = range(2, max_k + 1)

    for k in k_values:
        # KMeans
        kmeans = cluster.KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(data)
        metrics['kmeans']['inertia'].append(kmeans.inertia_)
        if len(set(labels)) > 1:
            metrics['kmeans']['silhouette'].append(silhouette_score(data, labels))
            metrics['kmeans']['davies_bouldin'].append(davies_bouldin_score(data, labels))
            metrics['kmeans']['calinski_harabasz'].append(calinski_harabasz_score(data, labels))

        # Birch
        birch = cluster.Birch(n_clusters=k)
        labels = birch.fit_predict(data)
        if len(set(labels)) > 1:
            metrics['birch']['inertia'].append(birch.subcluster_centers_.shape[0])  # Approximation of inertia
            metrics['birch']['silhouette'].append(silhouette_score(data, labels))
            metrics['birch']['davies_bouldin'].append(davies_bouldin_score(data, labels))
            metrics['birch']['calinski_harabasz'].append(calinski_harabasz_score(data, labels))

        # Agglomerative Clustering
        agg = cluster.AgglomerativeClustering(n_clusters=k)
        labels = agg.fit_predict(data)
        if len(set(labels)) > 1:
            metrics['agglomerative']['silhouette'].append(silhouette_score(data, labels))
            metrics['agglomerative']['davies_bouldin'].append(davies_bouldin_score(data, labels))
            metrics['agglomerative']['calinski_harabasz'].append(calinski_harabasz_score(data, labels))

    # Tracé des métriques pour chaque modèle
    models = ['kmeans', 'birch', 'agglomerative']
    for model in models:
        plt.figure(figsize=(15, 10))

        # Elbow Method
        plt.subplot(2, 2, 1)
        if 'inertia' in metrics[model] and len(metrics[model]['inertia']) > 0:
            plt.plot(k_values, metrics[model]['inertia'], 'bo-', markersize=4)
            plt.xlabel('Number of clusters, k')
            plt.ylabel('Inertia')
            plt.title(f'{model.capitalize()} - Elbow Method')
            plt.grid(True)
        else:
            plt.title(f'{model.capitalize()} - Elbow Method (No Data)')

        # Silhouette Score
        plt.subplot(2, 2, 2)
        if len(metrics[model]['silhouette']) > 0:
            plt.plot(k_values, metrics[model]['silhouette'], 'go-', markersize=4)
            plt.xlabel('Number of clusters, k')
            plt.ylabel('Silhouette Score')
            plt.title(f'{model.capitalize()} - Silhouette Score')
            plt.grid(True)
        else:
            plt.title(f'{model.capitalize()} - Silhouette Score (No Data)')

        # Davies-Bouldin Score
        plt.subplot(2, 2, 3)
        if len(metrics[model]['davies_bouldin']) > 0:
            plt.plot(k_values, metrics[model]['davies_bouldin'], 'ro-', markersize=4)
            plt.xlabel('Number of clusters, k')
            plt.ylabel('Davies-Bouldin Score')
            plt.title(f'{model.capitalize()} - Davies-Bouldin Score')
            plt.grid(True)
        else:
            plt.title(f'{model.capitalize()} - Davies-Bouldin Score (No Data)')

        # Calinski-Harabasz Score
        plt.subplot(2, 2, 4)
        if len(metrics[model]['calinski_harabasz']) > 0:
            plt.plot(k_values, metrics[model]['calinski_harabasz'], 'mo-', markersize=4)
            plt.xlabel('Number of clusters, k')
            plt.ylabel('Calinski-Harabasz Score')
            plt.title(f'{model.capitalize()} - Calinski-Harabasz Score')
            plt.grid(True)
        else:
            plt.title(f'{model.capitalize()} - Calinski-Harabasz Score (No Data)')

        plt.tight_layout()
        plt.suptitle(f'{model.capitalize()} Clustering Metrics', fontsize=16)
        plt.show()

# Exemple d'utilisation avec des données fictives
# data = ... (votre jeu de données)
plot_metrics_for_each_model(rfmls_sample_train)


Correspondance avec les sagments defini dans l'EDA
segment_order = ['Champions', 'Loyaux', 'Loyalistes potentiels', 'À réactiver', 'À risque', 'Perdus']

3 Correspond au 3 persona les plus presents et 6 correspond a tous les personas !

#### Visualisation des clusters

##### Vue 2D

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans


# Appliquer KMeans
kmeans = KMeans(n_clusters=3, random_state=42)
rfmls_sample['Cluster'] = kmeans.fit_predict(rfmls_sample[['R', 'F', 'M']])

# Analyser les centres des clusters
centers = pd.DataFrame(kmeans.cluster_centers_, columns=['R', 'F', 'M'])
# print("Centres des clusters :")
# print(centers)

# Calculer les statistiques descriptives pour chaque cluster
cluster_stats = rfmls_sample.groupby('Cluster')[['R', 'F', 'M']].agg(['mean', 'median', 'std'])
# print("Statistiques descriptives des clusters :")
# print(cluster_stats)

# Visualiser les clusters
plt.figure(figsize=(15, 5))

# Graphique Récence vs Fréquence
plt.subplot(1, 3, 1)
for cluster in range(3):
    plt.scatter(rfmls_sample[rfmls_sample['Cluster'] == cluster]['R'], rfmls_sample[rfmls_sample['Cluster'] == cluster]['F'], label=f'Cluster {cluster}')
plt.xlabel('Récence')
plt.ylabel('Fréquence')
plt.title('Récence vs Fréquence')
plt.legend()

# Graphique Fréquence vs Monetary
plt.subplot(1, 3, 2)
for cluster in range(3):
    plt.scatter(rfmls_sample[rfmls_sample['Cluster'] == cluster]['F'], rfmls_sample[rfmls_sample['Cluster'] == cluster]['M'], label=f'Cluster {cluster}')
plt.xlabel('Fréquence')
plt.ylabel('Monetary')
plt.title('Fréquence vs Monetary')
plt.legend()

# Graphique Récence vs Monetary
plt.subplot(1, 3, 3)
for cluster in range(3):
    plt.scatter(rfmls_sample[rfmls_sample['Cluster'] == cluster]['R'], rfmls_sample[rfmls_sample['Cluster'] == cluster]['M'], label=f'Cluster {cluster}')
plt.xlabel('Récence')
plt.ylabel('Monetary')
plt.title('Récence vs Monetary')
plt.legend()

plt.tight_layout()
plt.show()


##### Vue 3D

In [ ]:
import plotly.express as px
import pandas as pd

# Supposons que 'data' est votre DataFrame RFM avec une colonne 'Cluster'
# data = pd.DataFrame(...)

# Créer un graphique 3D avec Plotly
fig = px.scatter_3d(rfmls_sample, x='R', y='F', z='M', color='Cluster',
                    labels={'R': 'Récence', 'F': 'Fréquence', 'M': 'Monetary', 'Cluster': 'Cluster'},
                    title='Clustering RFM en 3D')

# Afficher le graphique
fig.show()


# Entrainement du modèle selectioné

#### Entrainement Kmeans

In [ ]:
model_list['kmeans']

##### Sauvegarde du modèle

# Maintenace des Modèles

## Functions

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import (
    silhouette_score, 
    davies_bouldin_score, 
    adjusted_rand_score, 
    calinski_harabasz_score  # Ajout de cette métrique
)

def compute_and_store_rfm(dfs, scaler, initial_model, product_familly_res, period="M"):
    """
    Calcule et stocke les résultats RFM pour chaque période définie par 'period'.

    Paramètres :
        dfs (pd.DataFrame): DataFrame contenant les données avec les colonnes R, F, M et order_purchase_timestamp.
        scaler: Scaler pour normaliser les données RFM.
        initial_model: Modèle initial de clustering.
        product_familly_res (list): Liste pour stocker les résultats.
        period (str): Fréquence de découpage ('M' pour mois, 'W' pour semaine, 'Y' pour année).
    """
    # Définir les bornes temporelles selon la fréquence choisie
    min_date = dfs['order_purchase_timestamp'].min()
    max_date = dfs['order_purchase_timestamp'].max()
    date_range = pd.date_range(start=min_date, end=max_date, freq=period)

    for i in range(len(date_range) - 1):
        time_bound = date_range[i + 1]
        print(f"Traitement pour la période se terminant avant {time_bound}...")

        # Filtrer les données jusqu'à la date limite
        df_period = dfs[dfs['order_purchase_timestamp'] < time_bound]

        if df_period.empty:
            print(f"Période vide avant {time_bound}, sautée.")
            continue

        # Extraire les colonnes R, F, M
        rfm = df_period[['R', 'F', 'M']].copy()

        # Standardiser les données RFM
        rfm_scaled = scaler.transform(rfm)

        # Appliquer le modèle initial
        labels_before_fit = initial_model.predict(rfm_scaled)
        silh_before_fit = silhouette_score(rfm_scaled, labels_before_fit)
        db_before_fit = davies_bouldin_score(rfm_scaled, labels_before_fit)
        ch_before_fit = calinski_harabasz_score(rfm_scaled, labels_before_fit)  # Ajout ici

        # Ré-entraîner le modèle K-means
        fitted_model = KMeans(n_clusters=3, random_state=42).fit(rfm_scaled)
        labels_after_fit = fitted_model.predict(rfm_scaled)
        silh_after_fit = silhouette_score(rfm_scaled, labels_after_fit)
        db_after_fit = davies_bouldin_score(rfm_scaled, labels_after_fit)
        ch_after_fit = calinski_harabasz_score(rfm_scaled, labels_after_fit)  # Ajout ici

        # Calculer l'indice de Rand ajusté
        ari = adjusted_rand_score(labels_before_fit, labels_after_fit)

        # Stocker les résultats
        month_res = {
            'time_bound': time_bound,
            'dataframe': rfm,
            'prop_data_period': round((rfm.shape[0] / dfs.shape[0] * 100), 2),
            'labels_before_fit': labels_before_fit,
            'silh_before_fit': silh_before_fit,
            'db_before_fit': db_before_fit,
            'ch_before_fit': ch_before_fit,  # Ajout ici
            'fitted_model': fitted_model,
            'labels_after_fit': labels_after_fit,
            'silh_after_fit': silh_after_fit,
            'db_after_fit': db_after_fit,
            'ch_after_fit': ch_after_fit,  # Ajout ici
            'ari': ari
        }
        product_familly_res.append(month_res)

        # Afficher les résultats pour la période
        print(f"Période se terminant avant {time_bound}:")
        print(f"Proportion du jeu de données: {month_res['prop_data_period']}%")
        print(f"Indice de silhouette avant réentraînement: {silh_before_fit}")
        print(f"Indice Davies-Bouldin avant réentraînement: {db_before_fit}")
        print(f"Indice Calinski-Harabasz avant réentraînement: {ch_before_fit}")  # Ajout ici
        print(f"Indice de silhouette après réentraînement: {silh_after_fit}")
        print(f"Indice Davies-Bouldin après réentraînement: {db_after_fit}")
        print(f"Indice Calinski-Harabasz après réentraînement: {ch_after_fit}")  # Ajout ici
        print(f"Indice Rand ajusté: {ari}")
        print("-" * 150)
        print()


def calculate_ari_timelaps(product_familly_res):
    """
    Trace l'évolution de l'Indice Rand Ajusté (ARI) au cours du temps.

    Paramètres :
        product_familly_res (list): Liste contenant les résultats pour chaque période.
    """
    # Extraire les dates et les valeurs ARI
    dates = [res['time_bound'] for res in product_familly_res]
    ari_values = [res['ari'] for res in product_familly_res]

    # Tracer l'évolution de l'ARI
    plt.figure(figsize=(12, 6))
    plt.plot(dates, ari_values, marker='o', linestyle='-')
    plt.title('Évolution de l\'Indice Rand Ajusté (ARI) au cours du temps')
    plt.xlabel('Date')
    plt.ylabel('ARI')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
# # Exemple d'utilisation
# if __name__ == "__main__":
#     # Initialisation des résultats
#     product_familly_res = []

#     # Charger votre DataFrame (remplacez ceci par votre propre code de chargement)
#     # dfs = pd.read_csv("votre_fichier.csv")
#     # Assurez-vous que votre DataFrame contient les colonnes suivantes :
#     # 'R', 'F', 'M', 'order_purchase_timestamp'

#     # Exemple : Simuler un DataFrame
#     data = {
#         'customer_unique_id': [f'cust_{i}' for i in range(1000)],
#         'R': [i % 365 for i in range(1000)],  # Recency
#         'F': [i % 50 + 1 for i in range(1000)],  # Frequency
#         'M': [i % 1000 + 100 for i in range(1000)],  # Monetary
#         'order_purchase_timestamp': pd.date_range('2020-01-01', periods=1000, freq='D')
#     }
#     dfs = pd.DataFrame(data)

#     # Normalisation des données RFM
#     from sklearn.preprocessing import StandardScaler
#     scaler = StandardScaler()

#     # Initialiser le modèle de clustering initial
#     initial_model = KMeans(n_clusters=3, random_state=42)

#     # Découper automatiquement en périodes (par exemple, par mois)
#     compute_and_store_rfm(dfs, scaler, initial_model, product_familly_res, period="M")

#     # Calculer et tracer l'évolution de l'ARI
#     calculate_ari_timelaps(product_familly_res)

## ARI RFM

link to sklearn doc [here](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html#sklearn.metrics.adjusted_rand_score)

In [ ]:

rfm_segment = pd.read_excel('RFM.xlsx')

# Filter the dataset to include only relevant segments
rfm_segment = rfm_segment[rfm_segment['Segment'].isin(['Perdus', 'À réactiver', 'À risque'])]

### labels_true

In [ ]:
# Encode the 'Segment' column into numerical labels
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()
true_labels = lb.fit_transform(rfm_segment['Segment'])

### labels_pred

In [ ]:
# 10%  du df
rfm_segment_sample_1 = rfm_segment.sample(frac=0.1, random_state=42)

# Drop non-numeric columns and prepare the feature set
rfm_segment_sample_1_predict = rfm_segment_sample_1.drop(
    ['Segment', 'Unnamed: 0', 'customer_unique_id'], axis=1
).reset_index(drop=True)

# Use KMeans to predict cluster labels
predicted_labels = model_list['kmeans'].fit_predict(rfm_segment_sample_1_predict)

### Claculate ARI

In [ ]:
from sklearn.metrics import adjusted_rand_score

# Calculate the Adjusted Rand Index
ari = adjusted_rand_score(true_labels, predicted_labels)
print(f"Adjusted Rand Index: {ari}")

### Plot ARI score /mois

In [ ]:
# Initialisation des résultats
product_familly_res = []


scaler = StandardScaler()

# Initialiser le modèle de clustering initial
initial_model = KMeans(n_clusters=3, random_state=42)

# Découper automatiquement en périodes (par exemple, par mois)
compute_and_store_rfm(rfm_segment, scaler, initial_model, product_familly_res, period="M")

# Calculer et tracer l'évolution de l'ARI
calculate_ari_timelaps(product_familly_res)

## ARI RFM LS

link to sklearn doc [here](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html#sklearn.metrics.adjusted_rand_score)

In [ ]:

rfm_segment = pd.read_excel('RFM.xlsx')

# Filter the dataset to include only relevant segments
rfm_segment = rfm_segment[rfm_segment['Segment'].isin(['Perdus', 'À réactiver', 'À risque'])]

##### labels_true

In [ ]:
# Encode the 'Segment' column into numerical labels
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()
true_labels = lb.fit_transform(rfm_segment['Segment'])

##### labels_pred

In [ ]:
# 10%  du df
rfm_segment_sample_1 = rfm_segment.sample(frac=0.1, random_state=42)

# Drop non-numeric columns and prepare the feature set
rfm_segment_sample_1_predict = rfm_segment_sample_1.drop(
    ['Segment', 'Unnamed: 0', 'customer_unique_id'], axis=1
).reset_index(drop=True)

# Use KMeans to predict cluster labels
predicted_labels = model_list['kmeans'].fit_predict(rfm_segment_sample_1_predict)

##### Claculate ARI

In [ ]:
from sklearn.metrics import adjusted_rand_score

# Calculate the Adjusted Rand Index
ari = adjusted_rand_score(true_labels, predicted_labels)
print(f"Adjusted Rand Index: {ari}")

### Plot ARI score /mois